In [2]:
# ============================
# ECG Prediction Notebook
# ============================

import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler

# ----------------------------
# Step 1: Load the trained model
# ----------------------------
model = load_model("ecg_cnn_model.keras")

# ----------------------------
# Step 2: Load the saved scaler (same one used during training)
# ----------------------------
scaler = StandardScaler()   # make sure you saved it earlier with joblib.dump(scaler, "scaler.pkl")

# ----------------------------
# Step 3: Load new ECG data
# ----------------------------
# Example: new ECG test dataset (without labels)
new_df = pd.read_csv("ecg_test.csv", header=None)

# ----------------------------
# Step 4: Normalize using same scaler
# ----------------------------
new_X = scaler.transform(new_df.values)

# ----------------------------
# Step 5: Reshape to match CNN input
# ----------------------------
new_X = new_X[..., np.newaxis]   # shape: (samples, signal_length, 1)

# ----------------------------
# Step 6: Predict
# ----------------------------
predictions = model.predict(new_X)

# Convert probabilities → 0 or 1
predicted_labels = (predictions > 0.5).astype(int)

# ----------------------------
# Step 7: Show predictions
# ----------------------------
for i, p in enumerate(predicted_labels):
    print(f"ECG {i+1}: {'Abnormal' if p[0] == 1 else 'Normal'}")


NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.